### Traffic Monitoring
The following code snippet processes video files to detect and count cars crossing a specified line within a region of interest (ROI). It uses OpenCV for video processing and object tracking, and pandas for summarizing the results in a DataFrame. This script is useful for traffic monitoring and analysis.

#### Installing and Importing Necessary Libraries

In [1]:
# Install necessary packages
!pip install opencv-python
!pip install numpy
!pip install pandas
!pip install scipy

In [2]:
# Import necessary libraries
import cv2
import numpy as np
import pandas as pd
from collections import deque
from scipy.spatial import distance as dist

#### Explanations:

1. **Install and Import Libraries**:
   - `opencv-python`, `numpy`, `pandas`, and `scipy` are installed and imported to handle video processing, numerical operations, data manipulation, and spatial calculations.

2. **CentroidTracker Class**:
   - This class tracks the centroids of detected objects across video frames, handles object registration, deregistration, and updating of object centroids.

3. **process_and_count_cars Function**:
   - This function processes a video to detect and count cars crossing a specified line within a defined region of interest (ROI).
   - It captures video frames, processes them to detect motion, and uses the `CentroidTracker` to track objects and count cars.

4. **Process Videos**:
   - The script processes each video file in the specified directory, retrieves the video details, checks if it meets the required format, and if not, converts it to the correct format.
   - It generates a report listing the issues for each video and writes this report to a text file.

5. **Output the Results**:
   - The results, including the total number of cars and the number of cars per minute for each video, are stored in a pandas DataFrame and displayed.

This script effectively tracks and counts cars in video footage, providing valuable traffic data that can be analyzed for various purposes such as traffic management and urban planning.

In [3]:
class CentroidTracker:
    def __init__(self, maxDisappeared=50, maxDistance=50):
        """
        Initialize the centroid tracker.

        Parameters:
        maxDisappeared (int): Maximum number of consecutive frames an object can be marked as disappeared before it is deregistered.
        maxDistance (int): Maximum distance between centroids to associate an object.
        """
        self.nextObjectID = 0 # Initialize the next unique object ID
        self.objects = {} # Dictionary to store the objects' centroids
        self.disappeared = {} # Dictionary to store the number of consecutive frames an object has disappeared for
        self.maxDisappeared = maxDisappeared
        self.maxDistance = maxDistance

    def register(self, centroid):
        """
        Register a new object with the given centroid.

        Parameters:
        centroid (tuple): The centroid coordinates of the object.
        """
        self.objects[self.nextObjectID] = centroid # Assign the centroid to the next available object ID
        self.disappeared[self.nextObjectID] = 0 # Initialize the disappeared counter for the object
        self.nextObjectID += 1 # Increment the next object ID

    def deregister(self, objectID):
        """
        Deregister an object with the given ID.

        Parameters:
        objectID (int): The ID of the object to be deregistered.
        """
        del self.objects[objectID] # Remove the object from the objects dictionary
        del self.disappeared[objectID] # Remove the object from the disappeared dictionary

    def update(self, rects):
        """
        Update the tracker with the new bounding box rectangles.

        Parameters:
        rects (list): List of bounding box rectangles for detected objects.

        Returns:
        dict: Updated dictionary of objects and their centroids.
        """
        # If no bounding box rectangles are provided
        if len(rects) == 0:
          # Mark all existing objects as disappeared
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)
            return self.objects

        inputCentroids = np.zeros((len(rects), 2), dtype="int")
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            cX = int((startX + endX) / 2.0) # Calculate the centroid X coordinate
            cY = int((startY + endY) / 2.0) # Calculate the centroid Y coordinate
            inputCentroids[i] = (cX, cY)

        if len(self.objects) == 0:
            # Register each new centroid if no existing objects are being tracked
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])
        else:
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())
            D = dist.cdist(np.array(objectCentroids), inputCentroids) # Compute the distance between old and new centroids
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]
            usedRows = set()
            usedCols = set()

            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue
                if D[row, col] > self.maxDistance:
                    continue
                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0
                usedRows.add(row)
                usedCols.add(col)

            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            for row in unusedRows:
                objectID = objectIDs[row]
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)

            for col in unusedCols:
                self.register(inputCentroids[col])

        return self.objects

In [4]:
def process_and_count_cars(input_path, roi_coords, line_position):
    """
    Process a video to detect and count cars crossing a specified line within a region of interest (ROI).

    Parameters:
    input_path (str): Path to the input video file.
    roi_coords (tuple): Coordinates (x1, y1, x2, y2) defining the ROI.
    line_position (int): X-coordinate of the line within the ROI for counting cars.

    Returns:
    tuple: Total number of cars counted and the number of cars per minute.
    """
    # Capture video from file
    video = cv2.VideoCapture(input_path)
    initial_frame = None # Initialize the first frame for baseline comparison
    total_cars = 0 # Initialize total car count
    fps = video.get(cv2.CAP_PROP_FPS) # Get frames per second of the video
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT)) # Get total number of frames in the video
    duration = frame_count / fps # Calculate the duration of the video in seconds
    tracker = CentroidTracker(maxDisappeared=40, maxDistance=50) # Initialize the centroid tracker
    counted_ids = set() # Set to store the IDs of counted cars
    last_positions = {} # Dictionary to store the last positions of the objects

    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        # Define the region of interest (ROI) using the provided coordinates
        roi = frame[roi_coords[1]:roi_coords[3], roi_coords[0]:roi_coords[2]]
        # Convert the ROI to grayscale
        gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        # Apply Gaussian blur to reduce noise and smoothen the image
        blur_frame = cv2.GaussianBlur(gray_frame, (21, 21), 0)
        # Set the first captured frame as the baseline image
        if initial_frame is None:
            initial_frame = blur_frame
            continue
        # Compute the absolute difference between the baseline and the current frame
        delta_frame = cv2.absdiff(initial_frame, blur_frame)
        # Apply a binary threshold to the delta frame to create a binary image
        threshold_frame = cv2.threshold(delta_frame, 25, 255, cv2.THRESH_BINARY)[1]
        # Create a kernel for morphological operations (dilation and erosion)
        kernel = np.ones((5, 5), np.uint8)
        # Dilate the threshold image to fill gaps in the detected objects
        threshold_frame = cv2.dilate(threshold_frame, kernel, iterations=2)
        # Erode the threshold image to remove small noise
        threshold_frame = cv2.erode(threshold_frame, kernel, iterations=2)
        # Find contours in the threshold frame
        contours, _ = cv2.findContours(threshold_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # List to store bounding box rectangles for detected objects
        rects = []
        for c in contours:
            if cv2.contourArea(c) < 1000:
                continue
            (x, y, w, h) = cv2.boundingRect(c)
            rects.append((x, y, x + w, y + h))
        # Update the tracker with the new bounding box rectangles
        objects = tracker.update(rects)
        # Iterate over the tracked objects
        for (objectID, centroid) in objects.items():
            if objectID in last_positions:
                prev_position = last_positions[objectID]
                # Check if the object has crossed the counting line
                if prev_position[0] >= line_position and centroid[0] < line_position and objectID not in counted_ids:
                    counted_ids.add(objectID)
                    total_cars += 1
                    print(f"Car counted: ID {objectID} at position {centroid}")
            last_positions[objectID] = centroid
    # Release the video capture object
    video.release()
    # Close all OpenCV windows
    cv2.destroyAllWindows()
    # Calculate the number of cars per minute
    cars_per_minute = total_cars / (duration / 60)
    return total_cars, cars_per_minute

In [5]:
# Define the ROI coordinates (x1, y1, x2, y2) for the main street
roi_coords = (0, 290, 1040, 600)
# Define the line position for counting cars within the ROI
line_position = 200  # Adjust this based on the actual line position in the ROI
# Process the videos and get the counts
video_files = ['/content/drive/MyDrive/ISP_Final/Ex_1/Traffic_Laramie_1.mp4', '/content/drive/MyDrive/ISP_Final/Ex_1/Traffic_Laramie_2.mp4']
# Initialize a list to store the results
results = []
# Process each video file and get the car counts
for video_file in video_files:
  total_cars, cars_per_minute = process_and_count_cars(video_file, roi_coords, line_position)
  results.append((video_file, total_cars, cars_per_minute))

Car counted: ID 0 at position [198  92]
Car counted: ID 2 at position [196 101]
Car counted: ID 19 at position [197 106]
Car counted: ID 20 at position [198 110]
Car counted: ID 16 at position [196 229]
Car counted: ID 38 at position [199 137]
Car counted: ID 3 at position [194  95]
Car counted: ID 19 at position [196 117]
Car counted: ID 25 at position [194 115]
Car counted: ID 45 at position [191 110]


In [6]:
# Display the results
df = pd.DataFrame(results, columns=['Video File', 'Total Number of Cars', 'Cars per Minute'])
df

,Video File,Total Number of Cars,Cars per Minute
0,/content/drive/MyDrive/ISP_Final/Ex_1/Traffic_...,6,2.023381
1,/content/drive/MyDrive/ISP_Final/Ex_1/Traffic_...,4,2.271007
